**TAREA FINAL versión 2 (con tareas complementarias): Datos de vivienda - ETL**

**Fecha: 8 de febrero de 2021**

**Autor: Diego Senso González**

# TAREA FINAL: Datos de vivienda

<p> El negocio inmobiliario es uno de los motores de la economía de España, el carácter turístico de nuesta península hace que sean muchos los inversores extranjeros que se decidan por buscar una inversión de bajo riesgos en nuestras ciudades costeras o grandes urbes. 
Muchas de estas inversiones se hacen sobre grandes bolsas de inmuebles que deben ser analizados previamente para comprobar la rentabilidad del porfolio </p>

<!-- <p> En este caso vamos a trabajar con una tabla que contienen información de distintos inmuebles repartidos por una zona específica, sus carácterísticas y su precio </p>  -->

Todas las cuestiones se deben realizar sobre el conjunto de casos que representen viviendas ubicadas en zonas residenciales **(alta, media y baja densidad)**

**MUY IMPORTANTE:** En las otras prácticas he detectado colaboraciones involucrando varias personas y he sido flexible aunque a algunos os lo he mencionado en las correcciones, porque al final el trabajo de analista de datos es un trabajo colaborativo. Sin embargo, este trabajo es individual, así que cuidado con las colaboraciones.

Las variables de las que se compone el dataset son:

|NOMBRE VARIABLE|DESCRIPTOR|VALORES|
| --- | --- | --- |
|Order|Variable de identificación|1 a 2930|
|MS Zoning|Zona de ubicación de la vivienda|"A rural, C comercial, FV residencial flotante, I industrial, RH residencial alta densidad, RL residencial baja densidad, RM residencial media densidad"|
|Lot Frontage|Longitud de la fachada en pies||
|Lot Area|Superficie de la vivienda en pies cuadrados||
|Land Contour|Contorno del terreno circundante|"Lvl llano, Bnk Tipo bancal, HLS Ladera, Low Depresión"|
|Land Slope|Tipo de pendiente de la vivienda|" Gtl pendiente suave, Mod pendiente moderada, Sev fuerte pendiente"|
|Overall Qual|Grado de calidad de materiales y acabado de la vivienda|De 1 (Muy pobre) a 10 (Excelente)|
|Year Built|Año de construccion de la vivienda||
|Year Remod/Add|Año de última reforma de la vivienda||
|Mas Vnr Type|Tipo de revestimiento exterior|" BrkCmn Ladrillo normal, BrkFace Ladrillo visto, CBlock Bloque de cemento, None Ninguna, Stone Piedra "|
|Exter Qual|Calidad de revestimiento exterior|"Ex Excelente,Gd Bueno,TA Media,Fa Justo"|
|Bsmt Cond|Estado general del sótano|"Ex Excelente, Gd Bueno, TA Media, Fa Justo, Po Pobre,Ss sin sótano"|
|Total Bsmt SF|Superficie del sótano en pies cuadrados|
|Heating QC|Calidad de la calefacción|"Ex Excelente,Gd Bueno,TA Media,Fa Justo,Po Pobre"|
|Central Air|Aire acondicionado centralizado|"N No Y Sí"|
|Full Bath|Número de baños completo en planta||
|Half Bath|Número de aseos en planta||
|Bedroom AbvGr|Número de dormitorios en planta||
|Kitchen AbvGr|Número de cocinas en planta||
|Kitchen Qual|Calidad de cocinas|"Ex Excelente,Gd Bueno,TA Media,Fa Justo,Po Pobre"|
|TotRms AbvGrd|Número total de habitaciones excluidos los cuartos de baño||
|Garage Cars|Número de plazas de garaje||
|Garage Area|Superficie del garaje|||
|Garage Cond|Estado del garaje|"Ex Excelente,Gd Bueno,TA Media,Fa Justo,Po Pobre,Sg sin garaje"|
|Pool Area|Superficie de la piscina en pies cuadrados|
|Pool QC|Calidad de la piscina|"Ex Excelente,Gd Bueno,TA Media,Fa Justo,Sp no hay piscina"|
|Mo Sold|mes de venta||
|Yr Sold|año de venta||
|SalePrice|precio de venta en dólares||

Recomiendo al leer los datos, eliminar espacios de los nombres de las columnas, realiza un pequeño análisis inicial de los mismos. No olvides fijarte en los tipos de variables, que variables pueden tener tipos confundidos y corrige los. Sobre todo, trabaja con las fechas.

**NOTA:** Las tareas complementarias sirven para subir nota. El resto de preguntas valen igual y suman 10 puntos.

## Inicializar y cargar el contexto spark

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-us.apache.org/dist/spark/spark-2.4.7/spark-2.4.7-bin-hadoop2.7.tgz
!tar xf spark-2.4.7-bin-hadoop2.7.tgz #d
!pip install -q findspark 

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.7-bin-hadoop2.7"

import findspark
findspark.init()
from pyspark import SparkContext
sc = SparkContext()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

Importamos Numpy y Pandas para realizar algunas funciones suyas a la hora del análisis exploratorio de los datos.

In [2]:
import pandas as pd
import numpy as np

Cargamos el fichero CSV:

In [6]:
viviendas = sc.textFile("BDpracticafinalCSV.csv")

También se carga con pandas:

In [7]:
viviendas_pandas = pd.read_csv("BDpracticafinalCSV.csv", delimiter = ';', index_col = 0)

Carga del archivo tipo XLSX:

In [8]:
from pyspark.shell import sqlContext
from pyspark.sql import SparkSession

spark = SparkSession.builder \
      .master('local') \
      .appName('Planning') \
      .enableHiveSupport() \
      .config('spark.executor.memory', '2g') \
      .getOrCreate()

data = sqlContext.read("BDpracticafinal2021.xlsx").show()

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.4.7
      /_/

Using Python version 3.6.9 (default, Oct  8 2020 12:12:24)
SparkSession available as 'spark'.


TypeError: ignored

Observamos las diferentes variables del dataset y el tipo de cada una:

In [9]:
viviendas_pandas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2936 entries, 1 to 2901
Data columns (total 28 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   MS Zoning       2936 non-null   object 
 1   Lot Frontage    2444 non-null   float64
 2   Lot Area        2936 non-null   int64  
 3   Land Contour    2936 non-null   object 
 4   Land Slope      2936 non-null   object 
 5   Overall Qual    2936 non-null   int64  
 6   Year Built      2936 non-null   int64  
 7   Year Remod/Add  2936 non-null   int64  
 8   Mas Vnr Type    2913 non-null   object 
 9   Exter Qual      2936 non-null   object 
 10  Bsmt Cond       2936 non-null   object 
 11  Total Bsmt SF   2936 non-null   int64  
 12  Heating QC      2936 non-null   object 
 13  Central Air     2936 non-null   object 
 14  Full Bath       2936 non-null   int64  
 15  Half Bath       2936 non-null   int64  
 16  Bedroom AbvGr   2936 non-null   int64  
 17  Kitchen AbvGr   2936 non-null   i

Procedemos a eliminar la primera fila de los datos. Luego, separamos por ';'.

In [10]:
primera_fila_viviendas = viviendas.take(1)[0]
viviendas = viviendas.filter(lambda x: x != primera_fila_viviendas)
viviendas_new = viviendas.map(lambda x: x.split(";"))

Ahora, creamos los filtros para seleccionar las observaciones de cada una de las tres zonas que se piden.

In [12]:
RL = viviendas_new.filter(lambda x: x[1] in ('RL', 'rL', 'Rl'))
RH = viviendas_new.filter(lambda x: x[1] == 'RH')
RM = viviendas_new.filter(lambda x: x[1] == 'RM')

Se unen los diferentes datos de las zonas, y se observa cómo ha quedado brevemente.

In [13]:
union_RL_RH = RL.union(RH)
viviendas_final = union_RL_RH.union(RM)
viviendas_final.take(1)

[['1',
  'RL',
  '141',
  '31770',
  'Lvl',
  'Gtl',
  '6',
  '1960',
  '1960',
  'Stone',
  'TA',
  'Gd',
  '1080',
  'Fa',
  'Y',
  '1',
  '0',
  '3',
  '1',
  'TA',
  '7',
  '2',
  '528',
  'TA',
  '0',
  'Sp',
  '5',
  '2010',
  '215000']]

## Cuántas viviendas distintas encontramos en el dataset? ¿Se repite alguna? Tiene sentido que haya duplicadas? ¿Qué podemos hacer con las duplicadas?

Se procede a contar primero los registros que hay en estas tres zonas, para saber cuántas viviendas hay en los datos con los que se va a trabajar durante la práctica.

In [ ]:
viviendas_final.count()

2768

Ahora, gracias a una función de Python se puede contar el número de viviendas duplicadas en total.

In [ ]:
viviendas_pandas.duplicated().sum()

8

Observando el diccionario de variables, se puede extraer la conclusión de que sí es posible que se repitan observaciones y sin embargo ser viviendas diferentes. Esto es debido a que en el dataset no existe ninguna variable que se pueda entender como un valor único para cada observación, como por ejemplo un identificador de vivienda o unas coordenadas concretas. Podría darse el caso de que dos viviendas del mismo edificio o urbanización respondan a unas características similares. Por ello, se ha decidido continuar con los duplicados y no eliminarlos, además de que son pocos valores.

## ¿Podrías decirme el total de inmuebles y el precio medio (Sale Price) de cada zona (MS Zoning)?

Se ha procedido a escoger los precios de venta de cada una de las tres zonas, además de contar el número de observaciones para cada una de tres zonas. Zona RL:

In [ ]:
zona_RL = RL.map(lambda x: (x[28]))
zona_RL=zona_RL.filter(lambda x_int: x_int.isdigit())
zona_RL=zona_RL.map(lambda x: float(x))

zona_RL.mean(), zona_RL.count()

(191412.60684510757, 2279)

Zona RM:

In [ ]:
zona_RM = RM.map(lambda x: (x[28]))
zona_RM=zona_RM.filter(lambda x_int: x_int.isdigit())
zona_RM=zona_RM.map(lambda x: float(x))

zona_RM.mean(), zona_RM.count()

(126781.39393939397, 462)

Zona RH:

In [ ]:
zona_RH = RH.map(lambda x: (x[28]))
zona_RH=zona_RH.filter(lambda x_int: x_int.isdigit())
zona_RH=zona_RH.map(lambda x: float(x))

zona_RH.mean(), zona_RH.count()

(136419.77777777778, 27)

A la vista de los resultados, la zona de baja densidad (RL) cuenta con muchas más viviendas. Además, supone el mayor precio medio de las casas. La zona más barata de media es la de densidad media.

## Media de Total Bsmt SF por cada década de construcción calculada a partir de Year Built. 
## ¿Cuál es la decada de construcción con viviendas mejor acondicionadas para el frío (Heating QC)?

Se interpreta que se debe escoger el Total Bsmt SF, y agruparlo por década de construcción. Para ello, primero se define una función que convierte cada año en la década en que se encuentra.

In [ ]:
def get_decade(x):
    return x[0] + x[1] + x[2] + "0s"

A continuación, se aplica la función seleccionando la columna del año de construcción, y se selecciona y convierte en float la columna del indicador solicitado. Posteriormente, se muestran los resultados agrupados por cada década.

In [ ]:
decada_construccion = viviendas_final.map(lambda x: (get_decade(x[7]), [float(x[3])]))
decada_construccion.reduceByKey(lambda x,y: [x[0]+y[0]]).take(15)

[('1940s', [1397747.0]),
 ('1900s', [343871.0]),
 ('2000s', [6934543.0]),
 ('1970s', [3349636.0]),
 ('1920s', [1544611.0]),
 ('1910s', [955525.0]),
 ('1930s', [867047.0]),
 ('1880s', [115197.0]),
 ('1990s', [3458945.0]),
 ('2010s', [18754.0]),
 ('1890s', [132274.0]),
 ('1960s', [4346102.0]),
 ('1950s', [3884896.0]),
 ('1870s', [25092.0]),
 ('1980s', [1183432.0])]

Segunda parte de la pregunta. Se interpreta que se deben obtener las viviendas que tengan un acondicionamiento de excelente. Luego, mostrar cuántas casas con esa calificación existen en cada década (en la que son construidas). Para ello, se realiza un filtro, se seleccionan las columnas del año (aplicando la función para obtener la década) y la columna de la calificación del acondicionamiento contra el frío. Por último, se cuentan los registros por cada década.

In [ ]:
excelente = viviendas_final.filter(lambda x: (x[13]== "Ex"))
excelente = excelente.map(lambda x: (get_decade(x[7]), x[13]))
excelente.countByKey()

defaultdict(int,
            {'1880s': 2,
             '1890s': 4,
             '1900s': 12,
             '1910s': 38,
             '1920s': 51,
             '1930s': 40,
             '1940s': 56,
             '1950s': 98,
             '1960s': 88,
             '1970s': 68,
             '1980s': 28,
             '1990s': 235,
             '2000s': 641,
             '2010s': 2})

## ¿Cuáles son las 10 viviendas que se vendieron por un precio más elevado por metro cuadrado en el año 2009?

Se extraen todas las viviendas vendidas en el año 2009. Tras ello, se selecciona tanto el precio de venta como la superficie de cada una de las viviendas. Debido a que la superficie está en pies, se multiplica por un valor para pasarlo a metros cuadrados. Por último, se divide precio entre metros cuadrados y se seleccionan los diez valores más elevados.

In [ ]:
año_2009 = viviendas_final.filter(lambda x: x[27] == '2009')
viviendas_caras = año_2009.map(lambda x: (x[27], [float(x[28]), float(x[3]) * 0.093]))
viviendas_caras.map(lambda x: [x[0],x[1][0]/x[1][1]]).top(10)

[['2009', 1025.6410256410256],
 ['2009', 989.9988437969708],
 ['2009', 820.2006409891715],
 ['2009', 807.1186531124097],
 ['2009', 764.8489503328212],
 ['2009', 755.2483358934971],
 ['2009', 716.8458781362007],
 ['2009', 713.536722322531],
 ['2009', 710.4454685099846],
 ['2009', 700.339558573854]]

## Media anual por zonas del precio de venta y metros cuadrados.

Se pretende obtener la media del precio de venta y de los metros cuadrados por zona y por año. Para comenzar, se extraen con filtros los datos anuales para cada zona.

In [ ]:
RL2010 = RL.filter(lambda x: x[27] == '2010')
RL2009 = RL.filter(lambda x: x[27] == '2009')
RL2008 = RL.filter(lambda x: x[27] == '2008')
RL2007 = RL.filter(lambda x: x[27] == '2007')
RL2006 = RL.filter(lambda x: x[27] == '2006')

RM2010 = RM.filter(lambda x: x[27] == '2010')
RM2009 = RM.filter(lambda x: x[27] == '2009')
RM2008 = RM.filter(lambda x: x[27] == '2008')
RM2007 = RM.filter(lambda x: x[27] == '2007')
RM2006 = RM.filter(lambda x: x[27] == '2006')

RH2010 = RH.filter(lambda x: x[27] == '2010')
RH2009 = RH.filter(lambda x: x[27] == '2009')
RH2008 = RH.filter(lambda x: x[27] == '2008')
RH2007 = RH.filter(lambda x: x[27] == '2007')
RH2006 = RH.filter(lambda x: x[27] == '2006')

A continuación, se extrae de cada año el dato del precio de venta y se le aplica la media.

Media anual de precio para la zona de baja densidad (RL) en orden desde los años 2010 a 2006:

In [ ]:
RL2010.map(lambda x: int(x[28])).mean(), RL2009.map(lambda x: int(x[28])).mean(), RL2008.map(lambda x: int(x[28])).mean(), RL2007.map(lambda x: int(x[28])).mean(), RL2006.map(lambda x: int(x[28])).mean()

(184978.41897233203,
 190455.5508637236,
 189885.5851528385,
 193399.98014440428,
 194911.2819472616)

Media anual de precio para la zona de densidad media (RM) en orden desde los años 2010 a 2006:

In [ ]:
RM2010.map(lambda x: int(x[28])).mean(), RM2009.map(lambda x: int(x[28])).mean(), RM2008.map(lambda x: int(x[28])).mean(), RM2007.map(lambda x: int(x[28])).mean(), RM2006.map(lambda x: int(x[28])).mean()

(119681.70689655174,
 118759.86904761905,
 133336.1219512195,
 128947.62105263153,
 127502.63725490194)

Media anual de precio para la zona de alta densidad (RH) en orden desde los años 2010 a 2006:

In [ ]:
RH2010.map(lambda x: int(x[28])).mean(), RH2009.map(lambda x: int(x[28])).mean(), RH2008.map(lambda x: int(x[28])).mean(), RH2007.map(lambda x: int(x[28])).mean(), RH2006.map(lambda x: int(x[28])).mean()

(126280.0, 116206.8, 150333.33333333334, 147250.0, 140316.66666666666)

Ahora, se pasa a calcular la media de metros cuadrados para cada zona. Para ello a los datos anuales se les extrae la columna de la superficie y se le multiplica por 0.093 para convertir en metros cuadrados.

Media anual de metros cuadrados para la zona de baja densidad (RL) en orden desde los años 2010 a 2006:

In [ ]:
RL2010.map(lambda x: float(x[3]) * 0.093).mean(), RL2009.map(lambda x: float(x[3]) * 0.093).mean(), RL2008.map(lambda x: float(x[3]) * 0.093).mean(), RL2007.map(lambda x: float(x[3]) * 0.093).mean(), RL2006.map(lambda x: float(x[3]) * 0.093).mean()

(1012.4152766798421,
 1003.2304491362762,
 1048.747192139738,
 1060.7843555956679,
 1044.7782231237325)

Media anual de metros cuadrados para la zona de densidad media (RM) en orden desde los años 2010 a 2006:

In [ ]:
RM2010.map(lambda x: float(x[3]) * 0.093).mean(), RM2009.map(lambda x: float(x[3]) * 0.093).mean(), RM2008.map(lambda x: float(x[3]) * 0.093).mean(), RM2007.map(lambda x: float(x[3]) * 0.093).mean(), RM2006.map(lambda x: float(x[3]) * 0.093).mean()

(586.7161551724139,
 553.1329999999998,
 624.3203414634144,
 626.1014526315793,
 575.6143823529411)

Media anual de metros cuadrados para la zona de alta densidad (RH) en orden desde los años 2010 a 2006:

In [ ]:
RH2010.map(lambda x: float(x[3]) * 0.093).mean(), RH2009.map(lambda x: float(x[3]) * 0.093).mean(), RH2008.map(lambda x: float(x[3]) * 0.093).mean(), RH2007.map(lambda x: float(x[3]) * 0.093).mean(), RH2006.map(lambda x: float(x[3]) * 0.093).mean()

(753.8208, 647.8194, 766.94, 651.99975, 660.2535)

## ¿Podrías decirme el total de recaudación de las casas de revistimiento (Mas Vnr Type) de piedra con respecto a las de ladrillo? ¿Hay diferencia significativa?

Para contestar, será necesario extraer por separado las casas de piedra y de ladrillo. Tras ello, extraer el sumatorio para comparar qué casas recaudan más. Primero, se extraen las casa de cada tipo de revestimiento.

In [ ]:
piedra = viviendas_final.filter(lambda x: x[9] == 'Stone')
ladrillo = viviendas_final.filter(lambda x: x[9] in ('BrkFace', 'BrkCmn'))

A continuación, para cada tipo se selecciona el precio de venta.

In [ ]:
piedra = piedra.map(lambda x: (x[28]))
piedra=piedra.filter(lambda x_int: x_int.isdigit())
piedra=piedra.map(lambda x: int(x))

ladrillo = ladrillo.map(lambda x: (x[28]))
ladrillo=ladrillo.filter(lambda x_int: x_int.isdigit())
ladrillo=ladrillo.map(lambda x: int(x))

Se cuenta el número de casas de cada tipo. Son mayoría las de ladrillo.

In [ ]:
piedra.count(), ladrillo.count()

(230, 875)

Y se le aplica la suma para obtener el total de recaudación. Como se puede observar, es superior el dato de las de ladrillo. Parece sí ser una diferencia significativa.

In [ ]:
piedra.sum(), ladrillo.sum()

(60495477, 182918036)

Sin embargo, hacemos la media también para ver si esa diferencia es producida por el número de casas de cada tipo.

In [ ]:
piedra.mean(), ladrillo.mean()

(263023.81304347835, 209049.184)

Efectivamente, la diferencia es producida por el número más elevado de casas de ladrillo. Las casas de ladrillo recaudan más en total, pero la media del precio es más alta para las de piedra.

## ¿Cuánto son más caras las viviendas con 2 cocinas, con 2 o más plazas de garaje que las que tienen 1 cocina y 1 plaza de garaje? Comparar medias y cuartiles de ambos casos

La idea extraída es comparar el precio de venta de ambos tipos de viviendas entre sí. Por un lado, se filtran aquellas viviendas que tengan 2 cocinas y 2 o más plazas de garaje. Se ha observado que el mayor número de plazas de garaje existente es de 5.

In [ ]:
primer_data = viviendas_final.filter(lambda x: x[18] == '2' and x [21] in ('2', '3', '4', '5'))
primer_data = primer_data.map(lambda x: (x[28]))
primer_data=primer_data.filter(lambda x_int: x_int.isdigit())
primer_data=primer_data.map(lambda x: int(x))

Ahora se extraen las viviendas con las que se pretende comparar las primeras. Serán aquellas casas que tengan una cocina y 1 plaza de garaje únicamente.

In [ ]:
segundo_data = viviendas_final.filter(lambda x: x[18] == '1' and x [21] == '1')
segundo_data = segundo_data.map(lambda x: (x[28]))
segundo_data=segundo_data.filter(lambda x_int: x_int.isdigit())
segundo_data=segundo_data.map(lambda x: int(x))

Se desea conocer cuántas viviendas están en cada uno de los dos grupos. Por las características, parece lógico que el segundo grupo de viviendas sea mayoritaria.

In [ ]:
primer_data.count(), segundo_data.count()

(86, 755)

En el precio medio, lógicamente también, el primer grupo parece más caro.

In [ ]:
primer_data.mean(), segundo_data.mean()

(145124.27906976745, 128121.99337748351)

Y en el primer grupo, también parece haber más dispersión en el precio.

In [ ]:
primer_data.stdev(), segundo_data.stdev()

(40319.38889396249, 30283.73797627781)

## (COMPLEMENTARIA) Estudiar la relación entre el precio y el número de Garajes. (Recomiendo segmentar precio por cuartiles y estudiar el número de casos coincidentes)

Se va a proceder a extraer las viviendas de cada número de plazas de garaje y comparar las medias de los precios, para ver si existen diferencias reseñables entre unos grupos y otros.

En primer lugar, se observan los posibles valores del número de garajes existentes en el dataset. Como se puede observar, van de 0 a 5.

In [73]:
plazas_garaje = viviendas_final.map(lambda x: (x[21]))
plazas_garaje.distinct().take(10)

['4', '', '3', '0', '2', '1', '5']

Se realizan filtros para generar cada uno de los grupos y se selecciona la variable del precio de venta, pues será la que se comparará posteriomente.

In [96]:
cero = viviendas_final.filter(lambda x: (x[21] == "0")).map(lambda x: (int(x[28])))
uno = viviendas_final.filter(lambda x: (x[21]== "1")).map(lambda x: (int(x[28])))
dos = viviendas_final.filter(lambda x: (x[21]== "2")).map(lambda x: (int(x[28])))
tres = viviendas_final.filter(lambda x: (x[21]== "3")).map(lambda x: (int(x[28])))
cuatro = viviendas_final.filter(lambda x: (x[21]== "4")).map(lambda x: (int(x[28])))
cinco = viviendas_final.filter(lambda x: (x[21]== "5")).map(lambda x: (int(x[28])))

Se observa el número de viviendas por grupo. La más común, con diferencia, es la que cuenta con dos plazas. De cinco plazas, solo hay una.

In [100]:
cero.count(), uno.count(), dos.count(), tres.count(), cuatro.count(), cinco.count()

(147, 764, 1481, 358, 16, 1)

Se pasa a comparar la media del precio de cada grupo. El que cuenta con el dato más elevado es el de tres plazas, seguido por el de cuatro. Las más baratas son las que no cuentan con garaje. 

In [101]:
cero.mean(), uno.mean(), dos.mean(), tres.mean(), cuatro.mean(), cinco.mean()

(107023.63945578231,
 128011.65575916233,
 181890.09318028364,
 311831.4888268157,
 228748.6875,
 126500.0)

## (COMPLEMENTARIA) Las 10 viviendas con mejores servicios y mejor precio.

Se ha tratado de seleccionar aquellas viviendas que contaban con servicios calificados como excelentes en las diferentes variables que hacían referencia a aspectos de la casa. Algunas de esas variable se han excluído, dado que muchas de las casas no contaban con piscina o plaza de garaje, por ejemplo. Finalmente, se han seleccionado aquellas viviendas que cuentan con servicios excelentes en la valoración general, en el revestimiento, en la calefacción, y en la cocina.

Se han realizado varios filtros para seleccionar las viviendas que tuvieran una calificación de excelente en todos esos parámetros. Posteriormente, se ha seleccionado el precio de esas viviendas y su fecha de construcción.

Finalmente, se han ordenado las diez primeras en orden ascendente para ver cuáles son las viviendas con mejor precio (entendido como más baratas) y que cuentan con los servicios anteriormente enunciados como excelentes.

In [105]:
overall = viviendas_final.filter(lambda x: (x[6] == '10'))
mejores = overall.filter(lambda x: (x[10]== "Ex"))
mejores = mejores.filter(lambda x: (x[13]== "Ex"))
mejores = mejores.filter(lambda x: (x[19]== "Ex"))

mejores = mejores.map(lambda x: (float(x[28]), x[7]))

mejores.takeOrdered(10, key = lambda x: x[0])

[(160000.0, '2008'),
 (183850.0, '2008'),
 (184750.0, '2007'),
 (337500.0, '2006'),
 (385000.0, '2007'),
 (386250.0, '2003'),
 (392000.0, '2006'),
 (402861.0, '2009'),
 (419005.0, '2007'),
 (426000.0, '2007')]

### Pista:
#### Calcula las variables: 
- Número de servicios excelentes
- Número de servicios buenos
...

#### Tendréis que tener en cuenta también variables como número de baños Full Bath, cocinas Kitchen AbvGr o dormitorios Bedroom AbvGr. Generando por ejemplo (número de estas variables por encima de media)

#### Finalmente precio de venta